In [40]:
import time
import csv
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import warnings
from statistics import mean
import xgboost as xgb
from joblib import dump
from joblib import load

In [88]:
DataSet_="IPFIXRecords"
# Chargement des données d'entraînement du défenseur
X_train_defender = pd.read_csv('Dataset/'+DataSet_+'/X_train_defender.csv')
y_train_defender = pd.read_csv('Dataset/'+DataSet_+'/y_train_defender.csv')

# Chargement des données de test du défenseur
X_test_defender = pd.read_csv('Dataset/'+DataSet_+'/X_test_defender.csv')
y_test_defender = pd.read_csv('Dataset/'+DataSet_+'/y_test_defender.csv')

# Chargement des données de validation du défenseur
X_val_defender = pd.read_csv('Dataset/'+DataSet_+'/X_val_defender.csv')
y_val_defender = pd.read_csv('Dataset/'+DataSet_+'/y_val_defender.csv')

# Chargement des données d'entraînement de l'attaquant
X_train_attacker = pd.read_csv('Dataset/'+DataSet_+'/X_train_attacker.csv')
y_train_attacker = pd.read_csv('Dataset/'+DataSet_+'/y_train_attacker.csv')

# Chargement des données de test de l'attaquant
X_test_attacker = pd.read_csv('Dataset/'+DataSet_+'/X_test_attacker.csv')
y_test_attacker = pd.read_csv('Dataset/'+DataSet_+'/y_test_attacker.csv')

# Chargement des données de validation de l'attaquant
X_val_attacker = pd.read_csv('Dataset/'+DataSet_+'/X_val_attacker.csv')
y_val_attacker = pd.read_csv('Dataset/'+DataSet_+'/y_val_attacker.csv')

# Assurer que les formes sont correctes
print("Forme de X_train_attacker:", X_train_attacker.shape)
print("Shape of X_test_attacker:", X_test_attacker.shape)
print("Shape of X_val_attacker:", X_val_attacker.shape)
print("Shape of X_train_defender:", X_train_defender.shape)
print("Shape of X_test_defender:", X_test_defender.shape)
print("Shape of X_val_defender:", X_val_defender.shape)


encoder = OneHotEncoder()

# Adapter et transformer les étiquettes du défenseur
y_train_encoded_defender = encoder.fit_transform(y_train_defender)
y_val_encoded_defender = encoder.transform(y_val_defender)
y_test_encoded_defender = encoder.transform(y_test_defender)

y_train_encoded_defender = np.array(y_train_encoded_defender.toarray())
y_val_encoded_defender = np.array(y_val_encoded_defender.toarray())
y_test_encoded_defender = np.array(y_test_encoded_defender.toarray())
encoder = OneHotEncoder()
# Adapter et transformer les étiquettes de l'attaquant
y_train_encoded_attacker = encoder.fit_transform(y_train_attacker)
y_val_encoded_attacker = encoder.transform(y_val_attacker)
y_test_encoded_attacker = encoder.transform(y_test_attacker)
y_train_encoded_attacker = np.array(y_train_encoded_attacker.toarray())
y_val_encoded_attacker = np.array(y_val_encoded_attacker.toarray())
y_test_encoded_attacker = np.array(y_test_encoded_attacker.toarray())

print(y_train_encoded_attacker.shape)
scaler = StandardScaler()
X_train_scaled_attacker = scaler.fit_transform(X_train_attacker)
X_test_scaled_attacker = scaler.transform(X_test_attacker)
X_val_scaled_attacker = scaler.transform(X_val_attacker)
scaler = StandardScaler()
X_train_scaled_defender = scaler.fit_transform(X_train_defender)
X_test_scaled_defender = scaler.transform(X_test_defender)
X_val_scaled_defender = scaler.transform(X_val_defender)

Forme de X_train_attacker: (170542, 14)
Shape of X_test_attacker: (63165, 14)
Shape of X_val_attacker: (18950, 14)
Shape of X_train_defender: (170542, 14)
Shape of X_test_defender: (63164, 14)
Shape of X_val_defender: (18950, 14)
(170542, 17)


In [89]:
warnings.filterwarnings("ignore", category=UserWarning)
# Sélection des paramètres
nmbVoisinsDefenseur = 8
nmbVoisinsAttaquant = 3
# Modèle KNN pour le défenseur
knn_defender = KNeighborsClassifier(n_neighbors=nmbVoisinsDefenseur)
knn_defender.fit(X_train_defender, y_train_defender)

# Prédiction et rapport de classification pour le défenseur
y_pred_defender = knn_defender.predict(X_test_defender)
report_defender = classification_report(y_test_defender, y_pred_defender, output_dict=True)
print("Perfermance de modèle de substitution KNN",report_defender['weighted avg'])
dump(knn_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\knn_defender.joblib')

# Modèle KNN pour l'attaquant
knn_attacker = KNeighborsClassifier(n_neighbors=nmbVoisinsAttaquant)
knn_attacker.fit(X_train_attacker, y_train_attacker)

# Prédiction et rapport de classification pour l'attaquant
y_pred_attacker = knn_attacker.predict(X_test_attacker)
report_attacker = classification_report(y_test_attacker, y_pred_attacker, output_dict=True)
print("Perfermance de modèle de l'attaqaunt KNN",report_attacker['weighted avg'])

dump(knn_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\knn_attacker.joblib')

Perfermance de modèle de substitution KNN {'precision': 0.9715239381140494, 'recall': 0.9713444367044519, 'f1-score': 0.9711837849099246, 'support': 63164.0}
Perfermance de modèle de l'attaqaunt KNN {'precision': 0.973673759638278, 'recall': 0.9730546980131402, 'f1-score': 0.9728340102905737, 'support': 63165.0}


['Model\\IPFIXRecords\\AttackerModel\\knn_attacker.joblib']

In [90]:
warnings.filterwarnings("ignore", category=UserWarning)

nmbEstimatorsDefenseur = 200
nmbEstimatorsAttaquant = 100

# RandomForest pour Défenseur
RandomForest_defender = RandomForestClassifier(n_estimators=nmbEstimatorsDefenseur, n_jobs=-1, random_state=0)
RandomForest_defender.fit(X_train_defender, y_train_defender)
pred_defender = RandomForest_defender.predict(X_test_defender)
report_defender = classification_report(y_test_defender, pred_defender, output_dict=True)
print("Perfermance de modèle de substitution RF",report_defender['weighted avg'])


dump(RandomForest_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\RandomForest_defender.joblib')

# RandomForest pour l'attaquant
RandomForest_attacker = RandomForestClassifier(n_estimators=nmbEstimatorsAttaquant, n_jobs=-1, random_state=0)
RandomForest_attacker.fit(X_train_attacker, y_train_attacker)
pred_attacker = RandomForest_attacker.predict(X_test_attacker)
report_attacker = classification_report(y_test_attacker, pred_attacker, output_dict=True)
print("Perfermance de modèle de substitution RF",report_defender['weighted avg'])
dump(RandomForest_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\RandomForest_attacker.joblib')

Perfermance de modèle de substitution RF {'precision': 0.9770670883406994, 'recall': 0.9763314546260529, 'f1-score': 0.9763028771846382, 'support': 63164.0}
Perfermance de modèle de substitution RF {'precision': 0.9770670883406994, 'recall': 0.9763314546260529, 'f1-score': 0.9763028771846382, 'support': 63164.0}


['Model\\IPFIXRecords\\AttackerModel\\RandomForest_attacker.joblib']

In [91]:
warnings.filterwarnings("ignore", category=UserWarning)

input_shape_defender = X_train_defender.shape[1]
input_shape_attacker = X_train_attacker.shape[1]
nb_neurones_defender=512
nb_neurones_attacker=256

# DNN pour Défenseur
DNN_defender = keras.Sequential([
    keras.layers.Dense(nb_neurones_defender, activation='relu', input_shape=(input_shape_defender,)),
    keras.layers.Dense(nb_neurones_defender, activation='relu'),
    keras.layers.Dense(nb_neurones_defender, activation='relu'),
    keras.layers.Dense(17, activation='softmax')
])
DNN_defender.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'],)
history_defender = DNN_defender.fit(X_train_scaled_defender, y_train_encoded_defender, epochs=10, batch_size=100)

y_pred_defender_prob = DNN_defender.predict(X_test_scaled_defender)
y_pred_defender = np.argmax(y_pred_defender_prob, axis=1)
report_defender = classification_report(y_test_defender, y_pred_defender,output_dict=True)
print("Perfermance de modèle de substitution DNN",report_defender['weighted avg'])

tf.keras.models.save_model(DNN_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\DNN_defender.h5')

# DNN pour l'attaquant
DNN_attacker = keras.Sequential([
    keras.layers.Dense((nb_neurones_attacker), activation='relu', input_shape=(input_shape_attacker,)),
    keras.layers.Dense(nb_neurones_attacker, activation='relu'),
    keras.layers.Dense(nb_neurones_attacker, activation='relu'),
    keras.layers.Dense(17, activation='softmax')
])
DNN_attacker.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
history_attacker = DNN_attacker.fit(X_train_scaled_attacker, y_train_encoded_attacker, epochs=10, batch_size=100, validation_data=(X_val_scaled_attacker, y_val_encoded_attacker))

y_pred_attacker_prob = DNN_attacker.predict(X_test_scaled_attacker)
y_pred_attacker= np.argmax(y_pred_attacker_prob, axis=1)
report_attacker = classification_report(y_test_attacker, y_pred_attacker,output_dict=True)
print("Perfermance de modèle de l'attaqaunt DNN",report_attacker['weighted avg'])

tf.keras.models.save_model(DNN_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\DNN_attacker.h5')

Epoch 1/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.6946 - loss: 0.8230
Epoch 2/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.8740 - loss: 0.3928
Epoch 3/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.9037 - loss: 0.2976
Epoch 4/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 32s 19ms/step - accuracy: 0.9144 - loss: 0.2684
Epoch 5/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 32s 19ms/step - accuracy: 0.9198 - loss: 0.2405
Epoch 6/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.9250 - loss: 0.2250
Epoch 7/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 35s 20ms/step - accuracy: 0.9234 - loss: 0.2304
Epoch 8/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.9265 - loss: 0.2157
Epoch 9/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.9279 - loss: 0.2104
Epoch 10/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - accuracy: 0.9301 - loss: 0.2041
1974/1974 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step


Perfermance de modèle de substitution DNN {'precision': 0.9284195720467078, 'recall': 0.9224241656639858, 'f1-score': 0.922746983635982, 'support': 63164.0}
Epoch 1/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.6828 - loss: 0.8899 - val_accuracy: 0.8389 - val_loss: 0.4495
Epoch 2/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.8548 - loss: 0.4252 - val_accuracy: 0.8968 - val_loss: 0.3498
Epoch 3/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.8918 - loss: 0.3334 - val_accuracy: 0.8993 - val_loss: 0.2931
Epoch 4/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9061 - loss: 0.2938 - val_accuracy: 0.9188 - val_loss: 0.2481
Epoch 5/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.9170 - loss: 0.2637 - val_accuracy: 0.9250 - val_loss: 0.2198
Epoch 6/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.9222 - loss: 0.2461 - val_accuracy: 0.9306 - val_loss: 0.2213
Epoch 7/10
1706/1706 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - 

Perfermance de modèle de l'attaqaunt DNN {'precision': 0.9318843327407886, 'recall': 0.9257658513417241, 'f1-score': 0.9250018452645145, 'support': 63165.0}


In [92]:
warnings.filterwarnings("ignore", category=UserWarning)

#XGBoost pour Défenseur
# Définir les paramètres du modèle XGBoost
params_defender = {
    'objective': 'multi:softmax',  # objectif de classification multiclasse
    'num_class': 17,  # nombre de classes
    'eval_metric': 'merror'  # métrique d'évaluation (taux d'erreur de classification)
}
params_attacker  = {
    'objective': 'multi:softmax',  
    'num_class': 17,  
    'eval_metric': 'merror' 
}

# Convertir les données au format DMatrix pour XGBoost
dtrain_defender = xgb.DMatrix(X_train_defender, label=y_train_defender)
dval_defender = xgb.DMatrix(X_val_defender, label=y_val_defender)
dtest_defender = xgb.DMatrix(X_test_defender)

dtrain_attacker = xgb.DMatrix(X_train_attacker, label=y_train_attacker)
dval_attacker = xgb.DMatrix(X_val_attacker, label=y_val_attacker)
dtest_attacker = xgb.DMatrix(X_test_attacker)

n_Estimateur_attaquant = 100 #nombre d'estimateur
n_Estimateur_defenseur = 200 
XGBoost_defender = xgb.train(params=params_defender,dtrain=dtrain_defender,num_boost_round=n_Estimateur_defenseur,)
pred_defender = XGBoost_defender.predict(dtest_defender)

report_defender = classification_report(y_test_defender, pred_defender, output_dict=True)
print("Perfermance de modèle de substitution XGBoost",report_defender['weighted avg'])
dump(XGBoost_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\XGBoost_defender.joblib')



#XGBoost pour l'attaquant
XGBoost_attacker = xgb.train(params=params_attacker ,dtrain=dtrain_attacker,num_boost_round=n_Estimateur_attaquant,)
pred_attacker = XGBoost_attacker.predict(dtest_attacker)
report_attacker = classification_report(y_test_attacker, pred_attacker, output_dict=True)
print("Perfermance de modèle de l'attaqaunt XGBoost",report_attacker['weighted avg'])

dump(XGBoost_attacker, 'Model\\' + DataSet_ + '\\AttackerModel\\XGBoost_attacker.joblib')


Perfermance de modèle de substitution XGBoost {'precision': 0.9764921362563042, 'recall': 0.975033246786144, 'f1-score': 0.9751962790412176, 'support': 63164.0}
Perfermance de modèle de l'attaqaunt XGBoost {'precision': 0.9772050633615792, 'recall': 0.9765693026201219, 'f1-score': 0.9762148551076512, 'support': 63165.0}


['Model\\IPFIXRecords\\AttackerModel\\XGBoost_attacker.joblib']

In [93]:
warnings.filterwarnings("ignore", category=UserWarning)

fct_qualite_defenseur="entropy"
fct_qualite_attacker="gini"
DecisionTree_defender = DecisionTreeClassifier(criterion=fct_qualite_defenseur, random_state=0)
DecisionTree_defender.fit(X_train_defender, y_train_defender)
pred_defender = DecisionTree_defender.predict(X_test_defender)
report_defender = classification_report(y_test_defender, pred_defender, output_dict=True)
print("Perfermance de modèle de substitution DT",report_defender['weighted avg'])
dump(DecisionTree_defender, 'Model\\' + DataSet_ + '\\DefenderModel\\DecisionTree_defender.joblib')


DecisionTree_attacker = DecisionTreeClassifier(criterion=fct_qualite_attacker, random_state=0)
DecisionTree_attacker.fit(X_train_attacker, y_train_attacker)
pred_attacker = DecisionTree_attacker.predict(X_test_attacker)
report_attacker = classification_report(y_test_attacker, pred_attacker, output_dict=True)
print("Perfermance de modèle de l'attaqaunt DT",report_attacker['weighted avg'])
dump(DecisionTree_attacker, 'Model/IPFIXHome/AttackerModel/DecisionTree_attacker.joblib')


Perfermance de modèle de substitution DT {'precision': 0.9758887352761707, 'recall': 0.9751757330124755, 'f1-score': 0.9751653005530446, 'support': 63164.0}
Perfermance de modèle de l'attaqaunt DT {'precision': 0.9763667865776678, 'recall': 0.976031029842476, 'f1-score': 0.9757026243692485, 'support': 63165.0}


['Model/IPFIXHome/AttackerModel/DecisionTree_attacker.joblib']